In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from dask import dataframe as dapd

import os


from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

%matplotlib inline

In [2]:
path = './favorita-grocery-sales-forecasting'

perfix = '' 
file_name =  '' 

os.chdir(path)

In [3]:
# list files in Kaggle data folder

with os.scandir() as bag:
    for obj in bag:
        if obj.is_file():
            print(obj.name)

combined_i_unit.csv
combined_r_unit.csv
holidays_events.csv
items.csv
oil.csv
sample_submission.csv
stores.csv
test.csv
train.csv
transactions.csv


## Load datasets

In [4]:

df_train_csv = dapd.read_csv('train.csv', sep=',', header='infer') #.set_index('id') #, index_col= 0
# df_train_csv.set_index('id')
df_train_csv

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
npartitions=148,,,,,,
,int64,object,int64,int64,float64,float64
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [5]:
# View value types of each columns
df_train_csv.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 6 entries, id to onpromotion
dtypes: object(1), float64(2), int64(3)

In [6]:
#checking for null values
df_train_csv.isnull().sum()

Dask Series Structure:
npartitions=1
date          int64
unit_sales      ...
dtype: int64
Dask Name: dataframe-sum-agg, 4 graph layers

In [7]:
# There are some transactions in stores that are not listed in train dataset.
# Fore example, in 2014-04-01 and at store_nbr = 1 there are 1216 items in train datasets but in transaction dataset it is 1847
df_train_csv[((df_train_csv['date']=='2014-04-01') & (df_train_csv['store_nbr']==1)) ]

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
npartitions=148,,,,,,
,int64,object,int64,int64,float64,float64
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [8]:
# delete rows with null value in 'onpromotion' column, there is no NaN in 'onpromotion' column of test dataset

df_train_csv = df_train_csv.dropna(subset=['onpromotion']) #, inplace=True

In [9]:
df_train_csv

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
npartitions=148,,,,,,
,int64,object,int64,int64,float64,float64
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [10]:
# It is good to add 'unit' column to this df for diffrentiate real and integer values in unit sales 

df_items_csv = dapd.read_csv('items.csv', sep=',', header='infer')
df_items_csv

,item_nbr,family,class,perishable
npartitions=1,,,,
,int64,object,int64,int64
,...,...,...,...


In [11]:
#checking for null values
df_items_csv.isnull().sum()

Dask Series Structure:
npartitions=1
class         int64
perishable      ...
dtype: int64
Dask Name: dataframe-sum-agg, 4 graph layers

In [12]:
df_items_csv['class'].unique().shape

(dd.Scalar<size-ag..., dtype=int64>,)

In [13]:
# Store dataset

df_stores_csv = dapd.read_csv('stores.csv', sep=',', header='infer')
df_stores_csv

,store_nbr,city,state,type,cluster
npartitions=1,,,,,
,int64,object,object,object,int64
,...,...,...,...,...


In [14]:
# Transaction dataset - this dataset is un-usale since there is no data availabe for test dates.

df_transactions_csv = dapd.read_csv('transactions.csv', sep=',', header='infer') 
df_transactions_csv

,date,store_nbr,transactions
npartitions=1,,,
,object,int64,int64
,...,...,...


In [15]:
# Oil price dataset

df_oil_csv = dapd.read_csv('oil.csv', sep=',', header='infer') 
df_oil_csv

,date,dcoilwtico
npartitions=1,,
,object,float64
,...,...


In [16]:
#checking for null values
df_oil_csv.isnull().sum()

Dask Series Structure:
npartitions=1
date          int64
dcoilwtico      ...
dtype: int64
Dask Name: dataframe-sum-agg, 4 graph layers

In [17]:
# Fill NaN values with price in a day before. The first NaN is in not usable date.
df_oil_csv = df_oil_csv.fillna(method='ffill') #, inplace=True
df_oil_csv

,date,dcoilwtico
npartitions=1,,
,object,float64
,...,...


In [18]:
# holidays_events price dataset

df_he_csv = dapd.read_csv('holidays_events.csv', sep=',', header='infer') 
df_he_csv

,date,type,locale,locale_name,description,transferred
npartitions=1,,,,,,
,object,object,object,object,object,bool
,...,...,...,...,...,...


In [19]:
#checking for null values
df_he_csv = df_he_csv[df_he_csv['transferred'] == False] # Remove records that were transferred (=True)
df_he_csv = df_he_csv[['date', 'locale']] # type has been removed, since there are dublicated dates with different types
df_he_csv

,date,locale
npartitions=1,,
,object,object
,...,...


In [20]:
df_he_csv['locale'].unique()

Dask Series Structure:
npartitions=1
    object
       ...
Name: locale, dtype: object
Dask Name: unique-agg, 8 graph layers

In [21]:
# Remove duplicated holidays (dates) with priority of National --> Regional --> Local
df_he_national = df_he_csv[df_he_csv['locale']== 'National']
df_he_regional = df_he_csv[df_he_csv['locale']== 'Regional']
df_he_local = df_he_csv[df_he_csv['locale']== 'Local']

df_he = dapd.concat([df_he_national, df_he_regional, df_he_local]).drop_duplicates(subset=['date'], keep='first')

df_he

,date,locale
npartitions=1,,
,object,object
,...,...


# Remove Noises

In [22]:
# Removig earthquake dates (3 weeks) from dataset
lst_earthquak_date = pd.date_range('2016-04-17', '2016-05-08', freq='d').strftime('%Y-%m-%d').tolist()

df_train_csv_nn = df_train_csv[~df_train_csv['date'].isin(lst_earthquak_date)] #, inplace=True
df_train_csv_nn



,id,date,store_nbr,item_nbr,unit_sales,onpromotion
npartitions=148,,,,,,
,int64,object,int64,int64,float64,float64
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


# Joining datasets

In [23]:
df_train_items = dapd.merge(left=df_train_csv_nn, right=df_items_csv, on = 'item_nbr', how='left')
df_train_items

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable
npartitions=148,,,,,,,,,
,int64,object,int64,int64,float64,float64,object,int64,int64
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [24]:
df_trian_items_stores = dapd.merge(left=df_train_items, right=df_stores_csv, on = 'store_nbr', how='left')
df_trian_items_stores

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,type,cluster
npartitions=148,,,,,,,,,,,,,
,int64,object,int64,int64,float64,float64,object,int64,int64,object,object,object,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...


In [25]:
df_trian_items_stores_oil = dapd.merge(left=df_trian_items_stores, right=df_oil_csv, on = 'date', how='left')
df_trian_items_stores_oil

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,type,cluster,dcoilwtico
npartitions=148,,,,,,,,,,,,,,
,int64,object,int64,int64,float64,float64,object,int64,int64,object,object,object,int64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [26]:
df_trian_items_stores_oil_he = dapd.merge(left=df_trian_items_stores_oil, right=df_he, on = 'date', how='left')
df_trian_items_stores_oil_he

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,type,cluster,dcoilwtico,locale
npartitions=148,,,,,,,,,,,,,,,
,int64,object,int64,int64,float64,float64,object,int64,int64,object,object,object,int64,float64,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [27]:
df_trian_items_stores_oil_he.sample(frac=0.0001)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,type,cluster,dcoilwtico,locale
npartitions=148,,,,,,,,,,,,,,,
,int64,object,int64,int64,float64,float64,object,int64,int64,object,object,object,int64,float64,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [28]:
df_trian_items_stores_oil_he['locale'] = df_trian_items_stores_oil_he['locale'].fillna('Working') #, inplace=True

In [29]:
df_trian_items_stores_oil_he['dcoilwtico'] = df_trian_items_stores_oil_he['dcoilwtico'].fillna(method='ffill')

In [38]:
df_trian_items_stores_oil_he.assign(holiday = df_trian_items_stores_oil_he.apply(lambda x: 0 if x.locale=='Working' else 1,axis = 1, meta=('x', 'i8')))

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,type,cluster,dcoilwtico,locale,holiday
npartitions=148,,,,,,,,,,,,,,,,
,int64,object,int64,int64,float64,float64,object,int64,int64,object,object,object,int64,float64,object,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [41]:
df_trian_items_stores_oil_he['class'].unique().shape

(dd.Scalar<size-ag..., dtype=int64>,)

In [42]:
# slit dataset into two datasets by differentiating unit_sales with integer or real value 
r_unit = df_trian_items_stores_oil_he[df_trian_items_stores_oil_he['unit_sales']%1 != 0][['class', 'family']]
r_unit

,class,family
npartitions=148,,
,int64,object
,...,...
...,...,...
,...,...
,...,...


In [43]:
# Itmes' class with real value unit_sales belonge to below families
r_unit_family = r_unit['family'].unique()
r_unit_family

Dask Series Structure:
npartitions=1
    object
       ...
Name: family, dtype: object
Dask Name: unique-agg, 48 graph layers

In [ ]:
# Items with real value unit sales
class_lst = r_unit['class'].to_list()

df_runit = df_trian_items_stores_oil_he[df_trian_items_stores_oil_he['class'].isin(class_lst)].reset_index(drop=True)
df_runit.index.name = 'id'
df_runit

In [42]:
# Items with integer value unit sales
df_iunit = df_trian_items_stores_oil_he[~df_trian_items_stores_oil_he['class'].isin(r_unit['class'])].reset_index(drop=True)
df_iunit.index.name = 'id'
df_iunit['unit_sales'] = df_iunit['unit_sales'].astype('int')
df_iunit

,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,type,cluster,dcoilwtico,locale,holiday
id,,,,,,,,,,,,,,,
0,2014-04-01,1,103520,1,False,GROCERY I,1028,0,Quito,Pichincha,D,13,99.69,Regional,1
1,2014-04-01,1,103665,3,False,BREAD/BAKERY,2712,1,Quito,Pichincha,D,13,99.69,Regional,1
2,2014-04-01,1,105574,1,False,GROCERY I,1045,0,Quito,Pichincha,D,13,99.69,Regional,1
3,2014-04-01,1,105575,11,False,GROCERY I,1045,0,Quito,Pichincha,D,13,99.69,Regional,1
4,2014-04-01,1,105577,4,False,GROCERY I,1045,0,Quito,Pichincha,D,13,99.69,Regional,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90764957,2017-08-15,54,2089339,4,False,GROCERY I,1006,0,El Carmen,Manabi,C,3,47.57,Local,1
90764958,2017-08-15,54,2106464,1,True,BEVERAGES,1148,0,El Carmen,Manabi,C,3,47.57,Local,1
90764959,2017-08-15,54,2110456,192,False,BEVERAGES,1120,0,El Carmen,Manabi,C,3,47.57,Local,1


In [43]:
df_iunit.to_csv('combined_i_unit.csv')

In [44]:
df_runit.to_csv('combined_r_unit.csv')